In [161]:
import pandas as pd
from database import DataBaseManager

db_connection = DataBaseManager().conn

with db_connection, db_connection.cursor() as cur:
    cur.execute("select * from pallets.test_results;")
    db_data = cur.fetchall()

In [162]:
df = pd.DataFrame(db_data)
test_data = df[[0, 1, 2, 3, 4]].rename(columns={0: 'id', 1: 'width_test', 2: 'height_test', 3: 'length_test', 4: 'volume_test'})
test_data['width_test'] = test_data['width_test'].apply(lambda num: round(num, 4))
test_data['height_test'] = test_data['height_test'].apply(lambda num: round(num, 4))
test_data['length_test'] = test_data['length_test'].apply(lambda num: round(num, 4))
test_data['volume_test'] = test_data['volume_test'].apply(lambda num: round(num, 4))
test_data

,id,width_test,height_test,length_test,volume_test
0,1,82.7615,65.9236,91.5872,4.996936e+05
1,2,170.9746,140.8698,126.7226,3.052133e+06
2,3,68.3523,85.4477,211.8690,1.237430e+06
3,4,123.0000,71.3757,105.0000,9.218167e+05
4,5,143.7811,143.8543,98.1337,2.029752e+06
5,6,91.4302,72.9709,114.1556,7.616163e+05
6,7,88.4168,86.5301,79.1568,6.056061e+05
7,8,155.1143,133.4664,215.3245,4.457765e+06
8,9,166.7262,70.6198,118.1848,1.391528e+06


In [163]:
test_data['volume_test'].to_list()

[499693.6075,
 3052132.7543,
 1237430.093,
 921816.7136,
 2029752.1864,
 761616.2581,
 605606.0739,
 4457764.6908,
 1391528.3056]

In [164]:
test_data_dict = dict(zip(test_data['id'], zip(test_data['width_test'], test_data['height_test'], test_data['length_test'], test_data['volume_test'])))

In [165]:
base_data = pd.read_excel('base_data.xlsx')
base_data['volume_base'] = base_data['width_base'] * base_data['height_base'] * base_data['length_base']
base_data

,id,width_base,height_base,length_base,volume_base
0,1,80,64,91,465920
1,2,168,138,125,2898000
2,3,87,85,210,1552950
3,4,123,70,105,904050
4,5,141,140,98,1934520
5,6,89,90,113,905130
6,7,88,89,80,626560
7,8,153,133,213,4334337
8,9,167,70,117,1367730


In [166]:
base_data_dict = dict(zip(base_data['id'], zip(base_data['width_base'], base_data['height_base'], base_data['length_base'], base_data['volume_base'])))

In [167]:
for key in base_data_dict:
    if base_data_dict[key][3] * 0.92 <= test_data_dict[key][3] <= base_data_dict[key][3] * 1.08:
        for dimension in base_data_dict[key][:-1]:
            if any(test_dim for test_dim in test_data_dict[key][:-1] if dimension - 4 <= test_dim <= dimension + 4):
                continue
            else:
                test_data.loc[test_data['id'] == key, 'is_correct'] = 'not_correct'
    else:
        test_data.loc[test_data['id'] == key, 'is_correct'] = 'not_correct'
        continue

In [168]:
test_data['is_correct'] = test_data['is_correct'].fillna('correct')
test_data

,id,width_test,height_test,length_test,volume_test,is_correct
0,1,82.7615,65.9236,91.5872,4.996936e+05,correct
1,2,170.9746,140.8698,126.7226,3.052133e+06,correct
2,3,68.3523,85.4477,211.8690,1.237430e+06,not_correct
3,4,123.0000,71.3757,105.0000,9.218167e+05,correct
4,5,143.7811,143.8543,98.1337,2.029752e+06,correct
5,6,91.4302,72.9709,114.1556,7.616163e+05,not_correct
6,7,88.4168,86.5301,79.1568,6.056061e+05,correct
7,8,155.1143,133.4664,215.3245,4.457765e+06,correct
8,9,166.7262,70.6198,118.1848,1.391528e+06,correct


In [169]:
test_data.to_excel('test_results.xlsx', index=False)